# Robust Scaler:

- faz a padronização de acordo com um quantil
- performa bem para dados com outliers quando comparado ao StandardScaler
- parâmetros:
lower = quartil inferior = 0,25 <br>
upper = quartil superior = 0,75 <br>
withScaling = dimensiona os dados para o quartil (padrão = true) <br>
withCentering = centraliza os dados com a mediana antes de transformar (padrão = False) <br>

In [1]:
#imports

import findspark, pyspark
from pyspark.sql import SparkSession
findspark.init() # inicializa interface pyspark jupyter
spark = SparkSession.builder.appName("robustscaler").getOrCreate()
#inicializa sessão no spark com nome importação

In [3]:
carros = spark.read.csv("Carros.csv", header=True, inferSchema=True, sep=";")
carros.show(5)

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
only showing top 5 rows



In [4]:
from pyspark.ml.feature import RobustScaler

In [6]:
# Vector Assembler - Criação de coluna com vetor dos atributos

from pyspark.ml.feature import VectorAssembler
vecassembler = VectorAssembler(inputCols=["Consumo", "Cilindros", "Cilindradas"], outputCol="vetor")
# transform sempre gera um df
carros_vetor = vecassembler.transform(carros)
carros_vetor.select("Consumo", "Cilindros", "Cilindradas","vetor").show()

+-------+---------+-----------+------------------+
|Consumo|Cilindros|Cilindradas|             vetor|
+-------+---------+-----------+------------------+
|     21|        6|        160|  [21.0,6.0,160.0]|
|     21|        6|        160|  [21.0,6.0,160.0]|
|    228|        4|        108| [228.0,4.0,108.0]|
|    214|        6|        258| [214.0,6.0,258.0]|
|    187|        8|        360| [187.0,8.0,360.0]|
|    181|        6|        225| [181.0,6.0,225.0]|
|    143|        8|        360| [143.0,8.0,360.0]|
|    244|        4|       1467|[244.0,4.0,1467.0]|
|    228|        4|       1408|[228.0,4.0,1408.0]|
|    192|        6|       1676|[192.0,6.0,1676.0]|
|    178|        6|       1676|[178.0,6.0,1676.0]|
|    164|        8|       2758|[164.0,8.0,2758.0]|
|    173|        8|       2758|[173.0,8.0,2758.0]|
|    152|        8|       2758|[152.0,8.0,2758.0]|
|    104|        8|        472| [104.0,8.0,472.0]|
|    104|        8|        460| [104.0,8.0,460.0]|
|    147|        8|        440|

In [9]:
robuster = RobustScaler(inputCol="vetor", outputCol="vetor_rbst",withScaling=True, withCentering=False,
                       lower=0.25, upper=0.75)
modelo = robuster.fit(carros_vetor)
carros_rbst = modelo.transform(carros_vetor)
carros_rbst.select("vetor","vetor_rbst").show(truncate=False)

+------------------+---------------------------------------------+
|vetor             |vetor_rbst                                   |
+------------------+---------------------------------------------+
|[21.0,6.0,160.0]  |[0.29166666666666663,1.5,0.16967126193001061]|
|[21.0,6.0,160.0]  |[0.29166666666666663,1.5,0.16967126193001061]|
|[228.0,4.0,108.0] |[3.1666666666666665,1.0,0.11452810180275717] |
|[214.0,6.0,258.0] |[2.972222222222222,1.5,0.27359490986214213]  |
|[187.0,8.0,360.0] |[2.597222222222222,2.0,0.3817603393425239]   |
|[181.0,6.0,225.0] |[2.513888888888889,1.5,0.23860021208907745]  |
|[143.0,8.0,360.0] |[1.986111111111111,2.0,0.3817603393425239]   |
|[244.0,4.0,1467.0]|[3.388888888888889,1.0,1.555673382820785]    |
|[228.0,4.0,1408.0]|[3.1666666666666665,1.0,1.4931071049840934]  |
|[192.0,6.0,1676.0]|[2.6666666666666665,1.5,1.7773064687168612]  |
|[178.0,6.0,1676.0]|[2.472222222222222,1.5,1.7773064687168612]   |
|[164.0,8.0,2758.0]|[2.2777777777777777,2.0,2.924708377518558]